# 경구약제 이미지 추가 데이터 EDA(TL1)

## 0. 전제
- 해당 EDA 는 추가 데이터셋을 다운로드하고, 해당하는 경로에 파일을 추가한 것을 전제로 진행하였습니다.

- 어노테이션(라벨링데이터-경구약제조합 5000종-TL_1_조합.zip) 경로: {프로젝트_루트}/data/added/train_images

- 이미지 경로(원천데이터-경구약제조합 5000종-TS_1_조합.zip)경로: {프로젝트_루트}/data/added/train_annotations  
    *원본 이미지는 기존의 어노테이션처럼 이미지 파일들이 폴더별로 구분되어 있습니다.

## 1. EDA 개요

추가를 고려중인(경구약제조합 5000종 TL_1_조합) 경구약제 이미지 데이터셋은 PNG 이미지와 COCO 포맷의 JSON 주석 파일로 구성되어 있습니다.  
본 EDA에서는 추가 데이터셋의 JSON 형식을 확인하여, 새로운 학습 데이터셋으로 추가 간에 문제가 없을지 파악합니다.

- 데이터셋 구조 및 규모 확인
- JSON 어노테이션 파일 구조 및 핵심 필드 이해
- 클래스(알약 종류) 정보 및 분포 분석
- 기존 클래스(56개)와 동일한 JSON 파일만 필터링
- 필터링된 JSON 파일과 매칭되는 이미지만을 지정

## 2. 환경 설정 및 라이브러리 임포트

필요한 라이브러리들을 임포트하고 데이터셋 경로를 설정합니다.

In [1]:
import os
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import random

# 경고 무시 (선택 사항)
import warnings
warnings.filterwarnings('ignore')

# Matplotlib 한글 폰트 설정
try:
    plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지
except Exception as e:
    print(f"경고: 한글 폰트 설정 중 오류 발생 ({e}). NanumGothic 폰트가 설치되어 있는지 확인해주세요.")

print("환경 설정 및 라이브러리 임포트 완료")

환경 설정 및 라이브러리 임포트 완료


In [2]:
# 데이터셋 경로 설정
DATA_ROOT = '../data/added' # 사용자 환경에 맞게 조정
TRAIN_IMG_DIR = os.path.join(DATA_ROOT, 'train_images')
TRAIN_ANNO_DIR = os.path.join(DATA_ROOT, 'train_annotations')

print(f"TRAIN_IMG_DIR: {TRAIN_IMG_DIR}")
print(f"TRAIN_ANNO_DIR: {TRAIN_ANNO_DIR}")

# 디렉토리 존재 여부 확인 및 입력 유도
if not os.path.exists(TRAIN_IMG_DIR) or not os.path.exists(TRAIN_ANNO_DIR):
    print("현재 설정된 경로 중 하나 이상을 찾을 수 없습니다.")
    
    suggested_root = input("올바른 DATA_ROOT 경로를 입력하세요: ")
    if os.path.exists(os.path.join(suggested_root, 'train_images')):
        DATA_ROOT = suggested_root
        TRAIN_IMG_DIR = os.path.join(DATA_ROOT, 'train_images')
        TRAIN_ANNO_DIR = os.path.join(DATA_ROOT, 'train_annotations')
        print(f"경로가 '{DATA_ROOT}'(으)로 업데이트 되었습니다. 계속 진행합니다.")
    else:
        print("입력된 경로도 유효하지 않아 프로그램을 종료합니다. 경로를 다시 확인해주세요.")
        exit() # 경로 문제로 EDA 진행 불가 시 종료

TRAIN_IMG_DIR: ../data/added\train_images
TRAIN_ANNO_DIR: ../data/added\train_annotations


## 3. 데이터셋 구조 및 규모 확인

주어진 파일 구조에 맞춰 train_annotations 내부의 .json 파일을 재귀적으로 탐색합니다.

In [5]:
# --- JSON 파일을 재귀적으로 탐색하는 헬퍼 함수 ---
def find_json_files_recursively(root_dir):
    json_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.json'):
                json_files.append(os.path.join(dirpath, filename))
    return json_files

# --- JSON과 동일하게 이미지도 재귀 탐색 ---
def find_files_recursively(root_dir, ext):
    files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(ext):
                files.append(os.path.join(dirpath, filename))
    return files

# 파일 개수 세기 (전체 경로 리스트로 변경)
train_image_files = find_files_recursively(TRAIN_IMG_DIR, '.png')
train_annotation_full_paths = find_json_files_recursively(TRAIN_ANNO_DIR)

print(f"총 훈련 이미지 개수: {len(train_image_files)}")
print(f"총 훈련 어노테이션 파일 개수: {len(train_annotation_full_paths)}")

if len(train_image_files) > 0 and len(train_annotation_full_paths) > 0:
    print("\n훈련 이미지와 어노테이션 파일이 모두 존재하는 것을 확인했습니다. 파싱 단계에서 정확한 매칭을 수행합니다.")
else:
    print("\n훈련 이미지 또는 어노테이션 파일이 충분하지 않아 보입니다. 확인이 필요합니다.")

총 훈련 이미지 개수: 582
총 훈련 어노테이션 파일 개수: 6011

훈련 이미지와 어노테이션 파일이 모두 존재하는 것을 확인했습니다. 파싱 단계에서 정확한 매칭을 수행합니다.


## 4. JSON 어노테이션 파일 분석

제공된 JSON 형식에 맞춰 images 섹션과 annotations 섹션에서 필요한 정보를 추출하고 Pandas DataFrame으로 통합합니다.  
images 섹션의 중복 처리 기준을 file_name으로 하여 정확한 이미지 메타데이터를 추출하고, 최종적으로 유효한 데이터만 필터링합니다.

In [6]:
all_images_meta = [] # 각 이미지의 메타데이터 (약 종류, 모양, 색상 등)
all_annotations = [] # 각 바운딩 박스 정보
all_category_mappings = [] # 모든 JSON 파일에서 발견된 category-id-name 매핑

print("추가된 훈련 어노테이션 파일 파싱 중...")
# 중복 image_file_name, annotation_id, category_id-name 쌍 처리를 위한 집합
parsed_image_filenames_set = set() # images['file_name'] 기준으로 중복 제거
parsed_annotation_ids_set = set()
parsed_category_id_name_pairs_set = set()

for full_path in tqdm(train_annotation_full_paths, desc="Parsing Annotations"):
    try:
        with open(full_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"경고: {full_path} 파일 파싱 중 오류 발생: {e}. 건너뜁니다.")
        continue
    except Exception as e:
        print(f"경고: {full_path} 파일 로드 중 예외 발생: {e}. 건너뜁니다.")
        continue

    # 'images' 섹션 처리
    if 'images' in data:
        for img_info in data['images']:
            if img_info['file_name'] not in parsed_image_filenames_set:
                img_info['source_json_file'] = full_path # 어떤 JSON 파일에서 왔는지 기록
                all_images_meta.append(img_info)
                parsed_image_filenames_set.add(img_info['file_name'])
    
    # 'annotations' 섹션 처리
    if 'annotations' in data:
        for anno_info in data['annotations']:
            # 'bbox' 유효성 검증 및 annotation ID 중복 방지
            if ('bbox' in anno_info and anno_info['bbox'] and len(anno_info['bbox']) == 4 and
                anno_info['id'] not in parsed_annotation_ids_set):
                anno_info['source_json_file'] = full_path
                all_annotations.append(anno_info)
                parsed_annotation_ids_set.add(anno_info['id'])

    # 'categories' 섹션 처리
    if 'categories' in data:
        for cat_info in data['categories']:
            # (id, name) 쌍으로 고유성 관리하여 중복 추가 방지
            cat_id_name_pair = (cat_info['id'], cat_info['name'])
            if cat_id_name_pair not in parsed_category_id_name_pairs_set:
                all_category_mappings.append(cat_info)
                parsed_category_id_name_pairs_set.add(cat_id_name_pair)

# DataFrame 생성
images_df = pd.DataFrame(all_images_meta)
annotations_df = pd.DataFrame(all_annotations)
categories_df = pd.DataFrame(all_category_mappings)

print("\n파싱 완료")
print(f"이미지 메타데이터 (images_df) 개수: {len(images_df)}")
print(f"어노테이션 (annotations_df) 개수: {len(annotations_df)}")
print(f"총 고유 카테고리 (categories_df) 개수: {len(categories_df)}")

# 어노테이션에 이미지 메타데이터 정보 병합
# `annotations_df`의 `image_id`와 `images_df`의 `id`를 기준으로 병합
annotations_df = pd.merge(annotations_df, 
                          images_df[['id', 'file_name', 'dl_name', 'drug_shape', 'color_class1', 
                                     'back_color', 'light_color', 'print_front', 'dl_idx']], 
                          left_on='image_id', right_on='id', how='left', suffixes=('_anno', '_img'))

# 중복된 'id_img' 컬럼은 제거 (annotations_df에 'image_id'가 이미 있으므로)
annotations_df.drop(columns=['id_img'], inplace=True)

# 최종 글로벌 카테고리 매핑 테이블 생성 (categories_df의 id와 name 사용)
global_categories_df = categories_df[['id', 'name']].copy()
global_categories_df.rename(columns={'id': 'global_category_id', 'name': 'global_class_name'}, inplace=True)

print(f"\n최종 통합된 고유 알약 클래스 개수 (global_categories_df): {len(global_categories_df)}개")

# 어노테이션 DataFrame에 최종 global_class_name 추가 (category_id는 이미 글로벌 ID와 동일함)
annotations_df = pd.merge(annotations_df, global_categories_df[['global_category_id', 'global_class_name']], 
                          left_on='category_id', right_on='global_category_id', how='left')
annotations_df.drop(columns=['global_category_id'], inplace=True) # 중복 컬럼 제거 (annotations_df에는 category_id가 이미 있음)

추가된 훈련 어노테이션 파일 파싱 중...


Parsing Annotations:   0%|          | 0/6011 [00:00<?, ?it/s]


파싱 완료
이미지 메타데이터 (images_df) 개수: 1503
어노테이션 (annotations_df) 개수: 1
총 고유 카테고리 (categories_df) 개수: 1

최종 통합된 고유 알약 클래스 개수 (global_categories_df): 1개


In [7]:
print("글로벌 카테고리 데이터프레임 헤드:")
global_categories_df.head()

글로벌 카테고리 데이터프레임 헤드:


,global_category_id,global_class_name
0,1,Drug


- 신규 Dataset 에서는 category_id 가 모두 1, class_name 은 'Drug' 로 통일되어 있음을 확인하였다.
- 즉, 추가된 데이터셋에서 category_id 와 class_name 은 'Drug' 에 대한 수정(기존 Train Dataset에 맞는 가공)이 필요함을 확인하였다.

### Raw Data-train_annotation 예시 ###
project_root\data\train_annotations\K-001900-016548-019607-029451_json\K-001900
```JSON
{
    "images": [
        {
            "file_name": "K-001900-016548-019607-029451_0_2_0_2_70_000_200.png",
            "width": 976,
            "height": 1280,
            "imgfile": "K-001900-016548-019607-029451_0_2_0_2_70_000_200.png",
            "drug_N": "K-001900",
            "drug_S": "정상알약",
            "back_color": "연회색 배경",
            "drug_dir": "앞면",
            "light_color": "주백색",
            "camera_la": 70,
            "camera_lo": 0,
            "size": 200,
            "dl_idx": "1899",
            "dl_mapping_code": "K-001900",
            "dl_name": "보령부스파정 5mg",
            "dl_name_en": "Buspar Tab. 5mg Boryung",
            "img_key": "http://connectdi.com/design/img/drug/1Mxwka5v0lL.jpg",
            "dl_material": "부스피론염산염",
            "dl_material_en": "Buspirone Hydrochloride",
            "dl_custom_shape": "정제, 저작정",
            "dl_company": "보령제약(주)",
            "dl_company_en": "Boryung",
            "di_company_mf": "",
            "di_company_mf_en": "",
            "item_seq": 198700706,
            "di_item_permit_date": "19870323",
            "di_class_no": "[01170]정신신경용제",
            "di_etc_otc_code": "전문의약품",
            "di_edi_code": "641901280,A09302381",
            "chart": "이약은 양면볼록한 장방형의 흰색정제이다",
            "drug_shape": "장방형",
            "thick": 2.5,
            "leng_long": 8,
            "leng_short": 4.5,
            "print_front": "BSP",
            "print_back": "5",
            "color_class1": "하양",
            "color_class2": "",
            "line_front": "",
            "line_back": "",
            "img_regist_ts": "20070910",
            "form_code_name": "나정",
            "mark_code_front_anal": "",
            "mark_code_back_anal": "",
            "mark_code_front_img": "",
            "mark_code_back_img": "",
            "mark_code_front": "",
            "mark_code_back": "",
            "change_date": "20160825",
            "id": 34
        }
    ],
    "type": "instances",
    "annotations": [
        {
            "area": 35910,
            "iscrowd": 0,
            "bbox": [
                644,
                845,
                189,
                190
            ],
            "category_id": 1899,
            "ignore": 0,
            "segmentation": [],
            "id": 133,
            "image_id": 34
        }
    ],
    "categories": [
        {
            "supercategory": "pill",
            "id": 1899,
            "name": "보령부스파정 5mg"
        }
    ]
}
```

### Added Data-train_annotation 예시 ###
project_root\data\added\train_annotations\K-000250-000573-002483-006192_json\K-002483\K-000250-000573-002483-006192_0_2_0_2_70_000_200.json

```JSON
{
	"images": [
		{
			"file_name": "K-000250-000573-002483-006192_0_2_0_2_70_000_200.png",
			"width": 976,
			"height": 1280,
			"imgfile": "K-000250-000573-002483-006192_0_2_0_2_70_000_200.png",
			"drug_N": "K-002483",
			"drug_S": "정상알약",
			"back_color": "연회색 배경",
			"drug_dir": "앞면",
			"light_color": "주백색",
			"camera_la": 70,
			"camera_lo": 0,
			"size": 200,
			"dl_idx": "2482",
			"dl_mapping_code": "K-002483",
			"dl_name": "뮤테란캡슐 100mg",
			"dl_name_en": "Muteran Cap. 100mg",
			"img_key": "http://connectdi.com/design/img/drug/1M_4NHfUrnp.jpg",
			"dl_material": "아세틸시스테인",
			"dl_material_en": "Acetylcysteine",
			"dl_custom_shape": "경질캡슐제",
			"dl_company": "한화제약(주)",
			"dl_company_en": "Hanwha Pharma",
			"di_company_mf": "",
			"di_company_mf_en": "",
			"item_seq": 198801531,
			"di_item_permit_date": "19880629",
			"di_class_no": "[02220]진해거담제",
			"di_etc_otc_code": "일반의약품",
			"di_edi_code": "651600290,A15301201",
			"chart": "흰색의 분말이 충진된 상하 녹색의 경질캡슐제",
			"drug_shape": "장방형",
			"thick": 6.8,
			"leng_long": 19,
			"leng_short": 6.4,
			"print_front": "Hanwha MUC100",
			"print_back": "",
			"color_class1": "초록",
			"color_class2": "초록",
			"line_front": "",
			"line_back": "",
			"img_regist_ts": "20191226",
			"form_code_name": "경질캡슐제, 산제",
			"mark_code_front_anal": "",
			"mark_code_back_anal": "",
			"mark_code_front_img": "",
			"mark_code_back_img": "",
			"mark_code_front": "",
			"mark_code_back": "",
			"change_date": "20190627",
			"id": 1
		}
	],
	"type": "instances",
	"annotations": [
		{
			"area": 106384,
			"iscrowd": 0,
			"bbox": [92,666,218,488],
			"category_id": 1,
			"ignore": 0,
			"segmentation": [],
			"id": 1,
			"image_id": 1
		}
	],
	"categories": [
		{
			"supercategory": "pill",
			"id": 1,
			"name": "Drug"
		}
	]
}
```

- 신규 데이터에서는
categories.id = 1,
categories.name = 'Drug'
로 모두 통일된 것을 알수 있다.

- 기존 학습 데이터에서는
images.dl_idx == categories.id
images.dl_name == categories.name
이 동일하므로,

- **신규 데이터에서도 이 규칙을 적용하여 json 파일을 수정해준다면 데이터셋을 통합하여 사용할 수 있을 것이다.**

---

In [8]:
# 데이터셋 경로 설정
RAW_DATA_ROOT = '../data/added' # 사용자 환경에 맞게 조정
RAW_TRAIN_IMG_DIR = os.path.join(RAW_DATA_ROOT, 'train_images')
RAW_TRAIN_ANNO_DIR = os.path.join(RAW_DATA_ROOT, 'train_annotations')

print(f"TRAIN_IMG_DIR: {TRAIN_IMG_DIR}")
print(f"TRAIN_ANNO_DIR: {TRAIN_ANNO_DIR}")

# 디렉토리 존재 여부 확인 및 입력 유도
if not os.path.exists(TRAIN_IMG_DIR) or not os.path.exists(TRAIN_ANNO_DIR):
    print("현재 설정된 경로 중 하나 이상을 찾을 수 없습니다.")
    
    suggested_root = input("올바른 DATA_ROOT 경로를 입력하세요: ")
    if os.path.exists(os.path.join(suggested_root, 'train_images')):
        DATA_ROOT = suggested_root
        TRAIN_IMG_DIR = os.path.join(DATA_ROOT, 'train_images')
        TRAIN_ANNO_DIR = os.path.join(DATA_ROOT, 'train_annotations')
        print(f"경로가 '{DATA_ROOT}'(으)로 업데이트 되었습니다. 계속 진행합니다.")
    else:
        print("입력된 경로도 유효하지 않아 프로그램을 종료합니다. 경로를 다시 확인해주세요.")
        exit() # 경로 문제로 EDA 진행 불가 시 종료

TRAIN_IMG_DIR: ../data/added\train_images
TRAIN_ANNO_DIR: ../data/added\train_annotations


In [9]:
# RAW 데이터셋 경로 설정 (raw 형식 변수명으로 변경)
RAW_DATA_ROOT = '../data' # 사용자 환경에 맞게 조정
RAW_TRAIN_IMG_DIR = os.path.join(RAW_DATA_ROOT, 'train_images')
RAW_TRAIN_ANNO_DIR = os.path.join(RAW_DATA_ROOT, 'train_annotations')

print(f"RAW_TRAIN_IMG_DIR: {RAW_TRAIN_IMG_DIR}")
print(f"RAW_TRAIN_ANNO_DIR: {RAW_TRAIN_ANNO_DIR}")

RAW_TRAIN_IMG_DIR: ../data\train_images
RAW_TRAIN_ANNO_DIR: ../data\train_annotations


In [10]:
# --- JSON 파일을 재귀적으로 탐색하는 헬퍼 함수 ---
def find_json_files_recursively(root_dir):
    json_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.json'):
                json_files.append(os.path.join(dirpath, filename))
    return json_files

# 파일 개수 세기 (소문자 + raw_ prefix)
raw_train_image_files = [f for f in os.listdir(RAW_TRAIN_IMG_DIR) if f.endswith('.png')]
raw_train_annotation_full_paths = find_json_files_recursively(RAW_TRAIN_ANNO_DIR)

print(f"[raw] 총 훈련 이미지 개수: {len(raw_train_image_files)}")
print(f"[raw] 총 훈련 어노테이션 파일 개수: {len(raw_train_annotation_full_paths)}")

[raw] 총 훈련 이미지 개수: 651
[raw] 총 훈련 어노테이션 파일 개수: 1001


In [11]:
raw_all_images_meta = []        # 각 이미지의 메타데이터 (약 종류, 모양, 색상 등)
raw_all_annotations = []        # 각 바운딩 박스 정보
raw_all_category_mappings = []  # 모든 JSON 파일에서 발견된 category-id-name 매핑

print("[raw] 훈련 어노테이션 파일 파싱 중...")
# 중복 image_file_name, annotation_id, category_id-name 쌍 처리를 위한 집합
raw_parsed_image_filenames_set = set()   # images['file_name'] 기준으로 중복 제거
raw_parsed_annotation_ids_set = set()
raw_parsed_category_id_name_pairs_set = set()

for full_path in tqdm(raw_train_annotation_full_paths, desc="[raw] Parsing Annotations"):
    try:
        with open(full_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"[raw] 경고: {full_path} 파일 파싱 중 오류 발생: {e}. 건너뜁니다.")
        continue
    except Exception as e:
        print(f"[raw] 경고: {full_path} 파일 로드 중 예외 발생: {e}. 건너뜁니다.")
        continue

    # 'images' 섹션 처리
    if 'images' in data:
        for img_info in data['images']:
            if img_info['file_name'] not in raw_parsed_image_filenames_set:
                img_info['raw_source_json_file'] = full_path  # 어떤 JSON 파일에서 왔는지 기록
                raw_all_images_meta.append(img_info)
                raw_parsed_image_filenames_set.add(img_info['file_name'])

    # 'annotations' 섹션 처리
    if 'annotations' in data:
        for anno_info in data['annotations']:
            # 'bbox' 유효성 검증 및 annotation ID 중복 방지
            if (
                'bbox' in anno_info and anno_info['bbox']
                and len(anno_info['bbox']) == 4
                and anno_info['id'] not in raw_parsed_annotation_ids_set
            ):
                anno_info['raw_source_json_file'] = full_path
                raw_all_annotations.append(anno_info)
                raw_parsed_annotation_ids_set.add(anno_info['id'])

    # 'categories' 섹션 처리
    if 'categories' in data:
        for cat_info in data['categories']:
            # (id, name) 쌍으로 고유성 관리하여 중복 추가 방지
            cat_id_name_pair = (cat_info['id'], cat_info['name'])
            if cat_id_name_pair not in raw_parsed_category_id_name_pairs_set:
                raw_all_category_mappings.append(cat_info)
                raw_parsed_category_id_name_pairs_set.add(cat_id_name_pair)

# DataFrame 생성
raw_images_df = pd.DataFrame(raw_all_images_meta)
raw_annotations_df = pd.DataFrame(raw_all_annotations)
raw_categories_df = pd.DataFrame(raw_all_category_mappings)

print("\n[raw] 파싱 완료")
print(f"[raw] 이미지 메타데이터 (raw_images_df) 개수: {len(raw_images_df)}")
print(f"[raw] 어노테이션 (raw_annotations_df) 개수: {len(raw_annotations_df)}")
print(f"[raw] 총 고유 카테고리 (raw_categories_df) 개수: {len(raw_categories_df)}")

# 어노테이션에 이미지 메타데이터 정보 병합
# `raw_annotations_df`의 `image_id`와 `raw_images_df`의 `id`를 기준으로 병합
raw_annotations_df = pd.merge(
    raw_annotations_df,
    raw_images_df[['id', 'file_name', 'dl_name', 'drug_shape',
                   'color_class1', 'back_color', 'light_color', 'print_front', 'dl_idx']],
    left_on='image_id',
    right_on='id',
    how='left',
    suffixes=('_anno', '_img')
)

# 중복된 'id_img' 컬럼은 제거 (raw_annotations_df에 'image_id'가 이미 있으므로)
raw_annotations_df.drop(columns=['id_img'], inplace=True)

# 최종 글로벌 카테고리 매핑 테이블 생성 (raw_categories_df의 id와 name 사용)
raw_global_categories_df = raw_categories_df[['id', 'name']].copy()
raw_global_categories_df.rename(
    columns={'id': 'global_category_id', 'name': 'global_class_name'},
    inplace=True
)

print(f"\n[raw] 최종 통합된 고유 알약 클래스 개수 (raw_global_categories_df): {len(raw_global_categories_df)}개")

# 어노테이션 DataFrame에 최종 global_class_name 추가 (category_id는 이미 글로벌 ID와 동일함)
raw_annotations_df = pd.merge(
    raw_annotations_df,
    raw_global_categories_df[['global_category_id', 'global_class_name']],
    left_on='category_id',
    right_on='global_category_id',
    how='left'
)
raw_annotations_df.drop(columns=['global_category_id'], inplace=True)  # 중복 컬럼 제거

[raw] 훈련 어노테이션 파일 파싱 중...


[raw] Parsing Annotations:   0%|          | 0/1001 [00:00<?, ?it/s]


[raw] 파싱 완료
[raw] 이미지 메타데이터 (raw_images_df) 개수: 369
[raw] 어노테이션 (raw_annotations_df) 개수: 1001
[raw] 총 고유 카테고리 (raw_categories_df) 개수: 56

[raw] 최종 통합된 고유 알약 클래스 개수 (raw_global_categories_df): 56개


In [12]:
# raw 데이터셋에서 56개 클래스 메타데이터 구성
print(f"[raw] 파싱된 raw 고유 알약 클래스 개수: {len(raw_global_categories_df)}개")
print("[raw] raw_global_categories_df 예시:")
display(raw_global_categories_df.head())

[raw] 파싱된 raw 고유 알약 클래스 개수: 56개
[raw] raw_global_categories_df 예시:


,global_category_id,global_class_name
0,1899,보령부스파정 5mg
1,16547,가바토파정 100mg
2,19606,스토가정 10mg
3,29450,레일라정
4,33008,신바로정


In [13]:
# added 데이터에서 raw 56개 클래스에 해당하는 샘플만 필터링
# raw_global_categories_df.global_category_id 와 images_df.dl_idx 비교

# 1) raw 56개 클래스 ID 집합 (dl_idx 와 비교용, 문자열로 맞춤)
raw_target_ids = set(raw_global_categories_df['global_category_id'].astype(str))
print(f"[filter] raw 56개 클래스 ID 수: {len(raw_target_ids)}")

# 2) added images_df 에서 dl_idx 기준으로 필터
added_filtered_images_df = images_df[images_df['dl_idx'].astype(str).isin(raw_target_ids)].copy()
print(f"[filter] raw 56개 클래스에 해당하는 added 이미지 수: {len(added_filtered_images_df)}")

# 3) 해당 이미지들만 포함하는 added_annotations_df 생성
added_valid_image_ids_for_56 = set(added_filtered_images_df['id'])
added_filtered_annotations_df = annotations_df[annotations_df['image_id'].isin(added_valid_image_ids_for_56)].copy()
print(f"[filter] raw 56개 클래스에 해당하는 added 어노테이션 수: {len(added_filtered_annotations_df)}")

[filter] raw 56개 클래스 ID 수: 56
[filter] raw 56개 클래스에 해당하는 added 이미지 수: 156
[filter] raw 56개 클래스에 해당하는 added 어노테이션 수: 1503


In [14]:
# added용 글로벌 카테고리 테이블 생성 (dl_idx / dl_name 기반)
# added 데이터 안에서 실제 등장하는 클래스만 추출

added_class_map_df = (
    added_filtered_images_df[['dl_idx', 'dl_name']]
    .drop_duplicates()
    .rename(columns={'dl_idx': 'global_category_id', 'dl_name': 'global_class_name'})
)

added_class_map_df['global_category_id'] = added_class_map_df['global_category_id'].astype(int)

print(f"[added] raw 56개 중, added 데이터에 실제 등장하는 클래스 수: {len(added_class_map_df)}개")
display(added_class_map_df.head())

[added] raw 56개 중, added 데이터에 실제 등장하는 클래스 수: 3개


,global_category_id,global_class_name
1347,1899,보령부스파정 5mg
1358,3543,무코스타정(레바미피드)(비매품)
1418,4542,에어탈정(아세클로페낙)


In [15]:
# added 어노테이션에 added_global_class_name 추가
# image_id -> dl_idx -> 클래스 이름 붙이기

# image_id -> dl_idx 매핑
added_image_id_to_dl_idx = (
    added_filtered_images_df.set_index('id')['dl_idx']
    .astype(int)
    .to_dict()
)

# annotations 쪽에 dl_idx 컬럼 추가
added_filtered_annotations_df['added_dl_idx'] = added_filtered_annotations_df['image_id'].map(added_image_id_to_dl_idx)

# dl_idx 기준으로 클래스 이름 붙이기
added_filtered_annotations_df = pd.merge(
    added_filtered_annotations_df,
    added_class_map_df[['global_category_id', 'global_class_name']],
    left_on='added_dl_idx',
    right_on='global_category_id',
    how='left'
)

print("[added] 필터링된 어노테이션 데이터프레임 헤드:")
display(added_filtered_annotations_df.head())

[added] 필터링된 어노테이션 데이터프레임 헤드:


,area,iscrowd,bbox,category_id,ignore,segmentation,id_anno,image_id,source_json_file,file_name,...,drug_shape,color_class1,back_color,light_color,print_front,dl_idx,global_class_name_x,added_dl_idx,global_category_id,global_class_name_y
0,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-006192_0_2_0_2_70_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
1,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-006192_0_2_0_2_75_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
2,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-006192_0_2_0_2_90_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
3,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-012778_0_2_0_2_70_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
4,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-012778_0_2_0_2_75_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg


In [16]:
display(added_filtered_annotations_df)

,area,iscrowd,bbox,category_id,ignore,segmentation,id_anno,image_id,source_json_file,file_name,...,drug_shape,color_class1,back_color,light_color,print_front,dl_idx,global_class_name_x,added_dl_idx,global_category_id,global_class_name_y
0,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-006192_0_2_0_2_70_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
1,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-006192_0_2_0_2_75_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
2,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-006192_0_2_0_2_90_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
3,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-012778_0_2_0_2_70_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
4,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-000250-000573-002483-012778_0_2_0_2_75_000_2...,...,원형,하양,연회색 배경,주백색,마크,249,Drug,1899,1899,보령부스파정 5mg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-001900-010224-016551-021026_0_2_0_2_75_000_2...,...,장방형,하양,연회색 배경,주백색,BSP,1899,Drug,1899,1899,보령부스파정 5mg
1499,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-001900-010224-016551-021026_0_2_0_2_90_000_2...,...,장방형,하양,연회색 배경,주백색,BSP,1899,Drug,1899,1899,보령부스파정 5mg
1500,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-001900-010224-016551-027926_0_2_0_2_70_000_2...,...,장방형,하양,연회색 배경,주백색,BSP,1899,Drug,1899,1899,보령부스파정 5mg
1501,75888,0,"[553, 184, 272, 279]",1,0,[],1,1,../data/added\train_annotations\K-000250-00057...,K-001900-010224-016551-027926_0_2_0_2_75_000_2...,...,장방형,하양,연회색 배경,주백색,BSP,1899,Drug,1899,1899,보령부스파정 5mg


In [17]:
# added COCO JSON용 category_id / name 재구성 (raw 규칙 적용)
# raw_global_categories_df (방금 파싱한 raw 카테고리 DF) 사용

# raw: global_category_id, global_class_name
raw_id_to_name = dict(
    zip(
        raw_global_categories_df['global_category_id'].astype(int),
        raw_global_categories_df['global_class_name']
    )
)

# added용 COCO categories 섹션 구성 (id = dl_idx, name = raw 쪽 이름)
added_coco_categories = []
for cid in sorted(added_class_map_df['global_category_id'].unique()):
    if cid in raw_id_to_name:
        added_coco_categories.append({
            "supercategory": "pill",
            "id": int(cid),
            "name": raw_id_to_name[cid]
        })

print(f"[coco] added COCO categories 개수: {len(added_coco_categories)}")
added_coco_categories[:5]

[coco] added COCO categories 개수: 3


[{'supercategory': 'pill', 'id': 1899, 'name': '보령부스파정 5mg'},
 {'supercategory': 'pill', 'id': 3543, 'name': '무코스타정(레바미피드)(비매품)'},
 {'supercategory': 'pill', 'id': 4542, 'name': '에어탈정(아세클로페낙)'}]

In [18]:
# added COCO JSON용 images / annotations 섹션 구성-

# images 섹션: added_filtered_images_df 기준
added_coco_images = []
for _, row in added_filtered_images_df.iterrows():
    added_coco_images.append({
        "id": int(row['id']),
        "file_name": row['file_name'],
        "width": int(row['width']),
        "height": int(row['height']),
        # 필요시 메타데이터 유지
        "dl_idx": str(row['dl_idx']),
        "dl_name": row['dl_name'],
        "drug_shape": row.get('drug_shape', None),
        "color_class1": row.get('color_class1', None),
        "back_color": row.get('back_color', None),
        "light_color": row.get('light_color', None),
    })

print(f"[coco] added COCO images 개수: {len(added_coco_images)}")

[coco] added COCO images 개수: 156


In [19]:
# annotations 섹션: category_id 를 image의 dl_idx (raw id) 로 교체
added_coco_annotations = []

for _, row in added_filtered_annotations_df.iterrows():
    img_id = int(row['image_id'])
    if img_id not in added_image_id_to_dl_idx:
        continue

    new_cat_id = int(added_image_id_to_dl_idx[img_id])  # dl_idx == raw category id

    added_coco_annotations.append({
        "id": 1,  # annotation id 는 모두 1로 통일
        "image_id": img_id,
        "category_id": new_cat_id,
        "bbox": row['bbox'],
        "area": float(row['area']),
        "iscrowd": int(row.get('iscrowd', 0)),
        "ignore": int(row.get('ignore', 0)),
        "segmentation": row.get('segmentation', []),
    })

print(f"[coco] added COCO annotations 개수: {len(added_coco_annotations)}")

[coco] added COCO annotations 개수: 1503


In [20]:
# fixed 데이터셋 경로 설정

FIXED_DATA_ROOT = "../data/added/fixed"
FIXED_TRAIN_IMG_DIR = os.path.join(FIXED_DATA_ROOT, "train_images")
FIXED_TRAIN_ANNO_DIR = os.path.join(FIXED_DATA_ROOT, "train_annotations")

os.makedirs(FIXED_TRAIN_IMG_DIR, exist_ok=True)
os.makedirs(FIXED_TRAIN_ANNO_DIR, exist_ok=True)

print("FIXED_TRAIN_IMG_DIR:", FIXED_TRAIN_IMG_DIR)
print("FIXED_TRAIN_ANNO_DIR:", FIXED_TRAIN_ANNO_DIR)

FIXED_TRAIN_IMG_DIR: ../data/added/fixed\train_images
FIXED_TRAIN_ANNO_DIR: ../data/added/fixed\train_annotations


In [ ]:
from pathlib import Path

# 1) 실제 존재하는 added 이미지의 상대 경로 집합 만들기
#    train_images/<조합>/<파일명>.png 에서 train_images/ 뒤를 상대 경로로 사용
existing_rel_paths = set()

for p in Path(TRAIN_IMG_DIR).rglob("*.png"):
    # p: ../data/added/train_images/K-조합/파일.png
    rel = p.relative_to(Path(TRAIN_IMG_DIR)).as_posix()  # "K-조합/파일.png"
    existing_rel_paths.add(rel)

print("실제 존재하는 added 이미지 개수:", len(existing_rel_paths))

# 2) added_filtered_images_df 에서 "조합/파일명" 형태의 상대 경로 생성
def make_rel_path(row):
    src_json = Path(row["source_json_file"]).resolve()
    combo_dir = src_json.parent.parent.name         # K-..._json
    combo_name = combo_dir.replace("_json", "")     # K-...
    fname = row["file_name"]                        # 파일명만
    return f"{combo_name}/{fname}"

added_filtered_images_df["rel_path"] = added_filtered_images_df.apply(make_rel_path, axis=1)

# 3) 실제 파일이 존재하는 샘플만 필터링
mask_exist = added_filtered_images_df["rel_path"].isin(existing_rel_paths)
added_filtered_images_df = added_filtered_images_df[mask_exist].copy()
print("실제로 이미지가 존재하는 added 이미지 수:", len(added_filtered_images_df))

# 4) 그에 맞춰 annotations 도 필터링
valid_image_ids = set(added_filtered_images_df["id"])
added_filtered_annotations_df = added_filtered_annotations_df[
    added_filtered_annotations_df["image_id"].isin(valid_image_ids)
].copy()
print("실제로 이미지가 존재하는 added 어노테이션 수:", len(added_filtered_annotations_df))

실제 존재하는 added 이미지 개수: 582
실제로 이미지가 존재하는 added 이미지 수: 0
실제로 이미지가 존재하는 added 어노테이션 수: 0


In [33]:
import shutil

ADDED_ORIG_TRAIN_IMG_DIR = TRAIN_IMG_DIR  # ../data/added/train_images

copied_cnt = 0
skipped_cnt = 0
missing_cnt = 0

for _, row in added_filtered_images_df.iterrows():
    rel_path = row["rel_path"]  # 이미 "K-조합/파일명.png" 형태로 만들어 둔 값

    src = os.path.join(ADDED_ORIG_TRAIN_IMG_DIR, rel_path)
    dst = os.path.join(FIXED_TRAIN_IMG_DIR, rel_path)

    if not os.path.exists(src):
        print(f"[copy warning] 원본 이미지 없음(필터 이후인데 이러면 구조 재점검 필요): {src}")
        missing_cnt += 1
        continue

    os.makedirs(os.path.dirname(dst), exist_ok=True)

    if os.path.exists(dst):
        skipped_cnt += 1
        continue

    shutil.copy2(src, dst)
    copied_cnt += 1

print(f"[copy] 새로 복사된 이미지: {copied_cnt}개, 이미 존재해서 건너뜀: {skipped_cnt}개, 원본 없음: {missing_cnt}개")


[copy] 새로 복사된 이미지: 0개, 이미 존재해서 건너뜀: 0개, 원본 없음: 0개


In [26]:
from pathlib import Path

# 그룹별 COCO JSON 저장 (비어있는 폴더는 생성하지 않음)

for src_json in tqdm(used_sources, desc="[fixed] Saving per-file COCO"):
    src_json_path = Path(src_json).resolve()

    # train_annotations/ 이후의 상대 경로 계산
    try:
        rel_path_from_train = src_json_path.relative_to(Path(TRAIN_ANNO_DIR).resolve())
    except ValueError:
        print(f"[skip] 예상 밖 경로 (TRAIN_ANNO_DIR 기준 아님): {src_json_path}")
        continue

    # 이 원본 JSON에서 온 이미지 / 어노테이션만 필터링
    img_mask = added_filtered_images_df["source_json_file"] == str(src_json_path)
    anno_mask = added_filtered_annotations_df["source_json_file"] == str(src_json_path)

    sub_images_df = added_filtered_images_df[img_mask].copy()
    sub_annos_df = added_filtered_annotations_df[anno_mask].copy()

    # 이 JSON에서 유효한 이미지/어노테이션이 하나도 없으면, 폴더/파일 아무 것도 생성하지 않음
    if len(sub_images_df) == 0 or len(sub_annos_df) == 0:
        continue

    # 여기서부터는 "실제로 JSON을 만들 이미지/어노테이션이 존재"하는 경우만
    # → 이제 폴더를 생성해도 됨
    fixed_json_path = Path(FIXED_TRAIN_ANNO_DIR).resolve() / rel_path_from_train
    fixed_json_path.parent.mkdir(parents=True, exist_ok=True)

    # 이미지 id -> dl_idx 매핑 (이 그룹 안에서만 사용)
    sub_image_id_to_dl_idx = (
        sub_images_df.set_index("id")["dl_idx"].astype(int).to_dict()
    )

    # images 섹션 구성
    coco_images = []
    for _, r in sub_images_df.iterrows():
        coco_images.append({
            "id": int(r["id"]),
            "file_name": r["file_name"],
            "width": int(r["width"]),
            "height": int(r["height"]),
            "dl_idx": str(r["dl_idx"]),
            "dl_name": r["dl_name"],
            "drug_shape": r.get("drug_shape", None),
            "color_class1": r.get("color_class1", None),
            "back_color": r.get("back_color", None),
            "light_color": r.get("light_color", None),
        })

    # annotations 섹션 구성 (annotation id 는 1로 통일)
    coco_annotations = []
    for _, r in sub_annos_df.iterrows():
        img_id = int(r["image_id"])
        if img_id not in sub_image_id_to_dl_idx:
            continue
        new_cat_id = int(sub_image_id_to_dl_idx[img_id])  # dl_idx == raw category id

        coco_annotations.append({
            "id": 1,
            "image_id": img_id,
            "category_id": new_cat_id,
            "bbox": r["bbox"],
            "area": float(r["area"]),
            "iscrowd": int(r.get("iscrowd", 0)),
            "ignore": int(r.get("ignore", 0)),
            "segmentation": r.get("segmentation", []),
        })

    # annotations 가 최종적으로 비어 있으면, 폴더도 이미 생성됐더라도 파일은 만들지 않음
    if len(coco_annotations) == 0:
        # 필요하면 빈 폴더 정리 로직을 추가할 수 있음
        continue

    # categories 섹션: 이 그룹 안의 dl_idx 들만 사용
    sub_class_ids = sorted(set(sub_image_id_to_dl_idx.values()))
    coco_categories = []
    for cid in sub_class_ids:
        if cid in raw_id_to_name:
            coco_categories.append({
                "supercategory": "pill",
                "id": int(cid),
                "name": raw_id_to_name[cid],
            })

    fixed_coco = {
        "info": {
            "description": "Added pill dataset (fixed per original JSON, category aligned to raw 56)",
            "version": "1.0",
        },
        "licenses": [],
        "images": coco_images,
        "annotations": coco_annotations,
        "categories": coco_categories,
    }

    with open(fixed_json_path, "w", encoding="utf-8") as f:
        json.dump(fixed_coco, f, ensure_ascii=False, indent=2)


[fixed] Saving per-file COCO:   0%|          | 0/157 [00:00<?, ?it/s]

### EDA 결과

- 신규 데이터셋에서 기존 카테고리 56개와 중복되는 어노테이션(json 파일)은 발견되었지만, 해당하는 이미지 파일은 발견되지 않았다.

- 따라서 해당 데이터셋에서 신규 도입 가능한 데이터는 없는 것으로 판단하였다.

- 추가로 다른 데이터셋에 동일한 코드를 실행하여 재검토 예정이다.